# Autocombat TOP and StrokeMRI vs. Insight harmonized datasets

Note this must be run in the `comscan6` environment

## import libraries

In [ ]:
import os
import sys

import pandas as pd
import numpy as np

sys.path.insert(0, '../../') # path to cvasl functions

import cvasl.harmony as har
import cvasl.seperated as sep
import cvasl.vendor.comscan.neurocombat as autocombat

## Let's see how we would apply this to our data
We will have to flip it on it's side to make it work

In [ ]:
MRI = pd.read_csv('../new_data/TrainingDataComplete_StrokeMRI.csv')
TOP = pd.read_csv('../new_data/TrainingDataComplete_TOP.csv')
Insight46 = pd.read_csv('../new_data/TrainingDataComplete_Insight46.csv')

In [ ]:
TOP = TOP.drop(['ID'], axis= 1)
TOP.head(3)

In [ ]:
MRI = MRI.drop(['ID'],axis = 1)
MRI.tail(5)

In [ ]:
Insight = Insight46.drop(['ID'],axis = 1)
Insight.tail(5)

In [ ]:
TOPMRI = pd.concat([TOP, MRI])
TOPMRI['Site'] = 0
TOPMRI.head(3) 

In [ ]:
datasets = [TOPMRI, Insight]
new_frames = sep.deal_with_readout_and_labelling(datasets, ['M0'])

In [ ]:
# make sure things are inproper order
for frame in new_frames:
    print(len(frame))
print(len(TOPMRI))
print(len(Insight))

In [ ]:
#new_frames[0]
TOPMRI = new_frames[0]
Insight = new_frames[1]


In [ ]:
new_frame_datasets=  [TOPMRI,  Insight, ]
for everyone in new_frame_datasets:
    everyone.columns =  everyone.columns.str.lower()
TOPMRI.head(3)

In [ ]:
TOPMRI.columns = TOPMRI.columns.str.lower()

In [ ]:
Insight.columns = Insight.columns.str.lower()

In [ ]:
TOPMRIInsight = pd.concat([TOPMRI, Insight])
TOPMRIInsight

In [ ]:
TOPMRIInsight['decade']=(TOPMRIInsight['age']/10).round()
#TOPMRI['decade']

In [ ]:
TOPMRIInsight = TOPMRIInsight.sort_values(by='age')
TOPMRIInsight.reset_index(inplace=True)
TOPMRIInsight['fine_grain'] = TOPMRIInsight['age'].rolling(2).sum()/2
TOPMRIInsight

In [ ]:
TOPMRIInsight[2:].fine_grain.iloc[::2]


In [ ]:
TOPMRIInsight[:1722].fine_grain.iloc[1::2]

In [ ]:
TOPMRIInsight[2:].fine_grain.iloc[::2] = TOPMRIInsight[:1722].fine_grain.iloc[1::2]
TOPMRIInsight['fine_grain']

In [ ]:
TOPMRIInsight['fine_grain'][0] = TOPMRIInsight['fine_grain'][1]
TOPMRIInsight['fine_grain'][1723] = TOPMRIInsight['fine_grain'][1722]
TOPMRIInsight['fine_grain']

In [ ]:
TOPMRIInsight['decade'].unique()

In [ ]:
TOPMRIInsight.columns

In [ ]:
combat = autocombat.Combat(
    features=[# 'gm_vol', 'wm_vol', 'csf_vol',
       #'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
        'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
        'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf',],
             #'ld', 'pld', 'labelling', 'readout',],
    sites=["site"], discrete_covariates=['sex'],continuous_covariates=['decade'],)

fg_combat = autocombat.Combat(
    features=[ #'gm_vol', 'wm_vol', 'csf_vol',
       #'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
       'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
       'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf',],
            # 'ld', 'pld', 'labelling', 'readout',],
    sites=[ "site"], discrete_covariates=['sex'],continuous_covariates=['fine_grain'],)


In [ ]:
print(combat.fit(TOPMRIInsight))
print(fg_combat.fit(TOPMRIInsight))

In [ ]:
print(combat.gamma_star_)
print(fg_combat.gamma_star_)

In [ ]:
transformed_TOPMRIInsight= combat.transform(TOPMRIInsight)
transformed_TOPMRIInsight.head(3)

In [ ]:
# check what harmonization did
for column in ['age', 'sex', 'site', 'gm_vol', 'wm_vol',
       'csf_vol', 'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
       'deepwm_b_cov', 'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
       'deepwm_b_cbf', 'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf',
       'ld', 'pld', 'labelling', 'readout', 'decade', 'fine_grain']:
        
    print(column, sum(TOPMRIInsight[column] - transformed_TOPMRIInsight[column]))

In [ ]:
fg_transformed_TOPMRIInsight= fg_combat.transform(TOPMRIInsight)
fg_transformed_TOPMRIInsight.head(3)

In [ ]:
TOPMRIInsight.head(3)

In [ ]:
transformed_TOPMRIInsight= combat.transform(TOPMRIInsight)
transformed_TOPMRIInsight.head(3)

In [ ]:
fg_transformed_TOPMRIInsight= fg_combat.transform(TOPMRIInsight)
fg_transformed_TOPMRIInsight.head(3)

In [ ]:
transformed_TOPMRIInsight.columns

In [ ]:
TOPMRI_transformed = transformed_TOPMRIInsight[ transformed_TOPMRIInsight['site']==0]
TOPMRI_transformed = TOPMRI_transformed.drop(['site', 'decade','fine_grain', 'index'], axis=1)
TOPMRI_transformed.head(3)

In [ ]:
fg_TOPMRI_transformed = fg_transformed_TOPMRIInsight[ fg_transformed_TOPMRIInsight['site']==0]
fg_TOPMRI_transformed = fg_TOPMRI_transformed .drop(['site', 'decade','fine_grain', 'index'], axis=1)
fg_TOPMRI_transformed.head(3)

In [ ]:
Insight_transformed = transformed_TOPMRIInsight[ transformed_TOPMRIInsight['site']==1]
Insight_transformed = Insight_transformed.drop(['site', 'decade', 'index', 'fine_grain'], axis=1)
Insight_transformed.head(3)

In [ ]:
fg_Insight_transformed = fg_transformed_TOPMRIInsight[fg_transformed_TOPMRIInsight['site']==1]
fg_Insight_transformed = fg_Insight_transformed.drop(['site', 'decade', 'index', 'fine_grain'], axis=1)
fg_Insight_transformed.head(3)

In [ ]:
TOPMRI_transformed.to_csv('harm_results/autocombat/autocom_harm_topmri_v_i.csv')
Insight_transformed.to_csv('harm_results/autocombat/autocom_harm_Insight.csv')

fg_TOPMRI_transformed.to_csv('harm_results/autocombat/fg_autocom_harm_topmri_v_i.csv')
fg_Insight_transformed.to_csv('harm_results/autocombat/fg_autocom_harm_Insight1.csv')

# Data quality check

In [ ]:
TOPMRI_transformed.isna().sum().sum()

In [ ]:
Insight_transformed.isna().sum().sum()

In [ ]:
number_columns = ['sex', 'gm_vol', 'wm_vol', 'csf_vol',
       'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
       'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
       'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf',]

In [ ]:
#neuro_harm_top.head(3)
(TOPMRI_transformed[number_columns] < 0).sum()

In [ ]:
(Insight_transformed[number_columns] < 0).sum()

In [ ]:
Insight_transformed

## So some variables are again problematic, becuas they are negative

 We will wait about logging it until we see other columns we may want to log across all datasets